In [1]:
# 데이터 준비

import pandas as pd
from sklearn.model_selection import train_test_split

wine = pd.read_csv('http://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy() # 입력 데이터
target = wine['class'].to_numpy() # 타깃 데이터

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size = 0.2, random_state = 42) # 훈련 세트와 테스트 세트
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size = 0.2, random_state = 42) # 훈련 세트와 검증 세트

In [3]:
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [5]:
# 모델 훈련

from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)

DecisionTreeClassifier(random_state=42)

In [6]:
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [7]:
# 교차 검증

from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)

In [8]:
print(scores)

{'fit_time': array([0.0090251 , 0.00778961, 0.00832367, 0.00844121, 0.0082252 ]), 'score_time': array([0.00124764, 0.00146365, 0.00125384, 0.00127554, 0.00125909]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [9]:
import numpy as np

print(np.mean(scores['test_score'])) # 교차 검증 최종 점수

0.855300214703487


In [10]:
# KFold 분할기

from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv = StratifiedKFold())
print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
# 10-폴드 교차 검증

splitter = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 42)
scores = cross_validate(dt, train_input, train_target, cv = splitter)

In [12]:
print(np.mean(scores['test_score']))

0.8574181117533719


In [14]:
# 그리드 서치

from sklearn.model_selection import GridSearchCV

params = {'min_impurity_decrease': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

dt = gs.best_estimator_ # 검증 점수가 가장 높은 모델

In [15]:
print(dt.score(train_input, train_target))

0.9615162593804117


In [16]:
print(gs.best_params_) # 최적의 매개변수

{'min_impurity_decrease': 0.0001}


In [17]:
print(gs.cv_results_['mean_test_score']) #각 매개변수에서 수행한 교차 검증의 평균 점수

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]


In [19]:
best_index = np.argmax(gs.cv_results_['mean_test_score']) # 가장 큰 값의 인덱스
print(gs.cv_results_['params'][best_index])

{'min_impurity_decrease': 0.0001}


In [27]:
params = {'min_impurity_decrease': np.arange(0.0001, 0.001, 0.0001),
          'max_depth': range(5, 20 ,1),
          'min_samples_split': range(2, 100, 10)
          }

In [28]:
gs = GridSearchCV(DecisionTreeClassifier(random_state = 42), params, n_jobs = -1)
gs.fit(train_input, train_target)

GridSearchCV(estimator=DecisionTreeClassifier(random_state=42), n_jobs=-1,
             param_grid={'max_depth': range(5, 20),
                         'min_impurity_decrease': array([0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008,
       0.0009]),
                         'min_samples_split': range(2, 100, 10)})

In [29]:
print(gs.best_params_)

{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [31]:
print(np.max(gs.cv_results_['mean_test_score'])) # 최상의 교차 검증 점수

0.8683865773302731


In [32]:
from scipy.stats import uniform,randint

rgen = randint(0, 10)
rgen.rvs(10)

array([5, 3, 4, 7, 9, 8, 4, 1, 3, 0])

In [33]:
np.unique(rgen.rvs(1000), return_counts = True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([116,  79,  95, 111, 100, 103, 101,  91, 109,  95]))

In [34]:
ugen = uniform(0, 1)
ugen.rvs(10)

array([0.08014522, 0.06789504, 0.93183598, 0.14684218, 0.04896433,
       0.25924657, 0.08586939, 0.17127671, 0.00434204, 0.37729474])

In [35]:
params = {'min_impurity_decrease': uniform(0.0001, 0.001),
          'max_depth': randint(20, 50),
          'min_samples_split': randint(2, 25),
          'min_samples_leaf': randint(1, 25)
          }

In [36]:
# 랜덤 서치

from sklearn.model_selection import RandomizedSearchCV

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state = 42), params, n_iter = 100, n_jobs = -1, random_state = 42)
gs.fit(train_input, train_target)

RandomizedSearchCV(estimator=DecisionTreeClassifier(random_state=42),
                   n_iter=100, n_jobs=-1,
                   param_distributions={'max_depth': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79b3082996c0>,
                                        'min_impurity_decrease': <scipy.stats._distn_infrastructure.rv_continuous_frozen object at 0x79b3086daa70>,
                                        'min_samples_leaf': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79b359223a00>,
                                        'min_samples_split': <scipy.stats._distn_infrastructure.rv_discrete_frozen object at 0x79b308299240>},
                   random_state=42)

In [38]:
print(gs.best_params_)

{'max_depth': 39, 'min_impurity_decrease': 0.00034102546602601173, 'min_samples_leaf': 7, 'min_samples_split': 13}


In [39]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8695428296438884


In [40]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))

0.8928227823744468
